In [6]:
import pandas as pd
import numpy as np
import os 
import sys
sys.path.append('../')

df = pd.read_csv("HIV_integrase.csv", index_col=None)
df.head(2)

,ID,Canomical_smiles,Activity
0,CHEMBL19464,O=C(O)C(=O)CC(=O)c1cccc(Cc2ccccc2)c1,8.00
1,CHEMBL193783,CCc1cc(/C=N/CC/N=C/c2cc(CC)sc2S)c(S)s1,5.22


# 1. Standardize

In [7]:
from standardize import standardization
from rdkit.rdBase import BlockLogs
block = BlockLogs()
std = standardization(data=df,ID='ID', smiles_col='Canomical_smiles', active_col='Activity', ro5 =4)
data = std.filter_data()
data.head(2)

  0%|          | 0/2445 [00:00<?, ?it/s]

  0%|          | 0/2377 [00:00<?, ?it/s]

,ID,Canomical_smiles,Activity,Canomicalsmiles,Molecule
0,CHEMBL19464,O=C(O)C(=O)CC(=O)c1cccc(Cc2ccccc2)c1,8.00,O=C(O)C(=O)CC(=O)c1cccc(Cc2ccccc2)c1,<rdkit.Chem.rdchem.Mol object at 0x7f068fe025f0>
1,CHEMBL193783,CCc1cc(/C=N/CC/N=C/c2cc(CC)sc2S)c(S)s1,5.22,CCc1cc(/C=N/CC/N=C/c2cc(CC)sc2S)c(S)s1,<rdkit.Chem.rdchem.Mol object at 0x7f068fe02660>


# 2. Medicinal chemistry filter

In [8]:
from medicinal_chemistry import *
from medicinal_chemistry import calculate_all, calculate_filters, calculate_ro5_properties
data_filter = df['Canomical_smiles'].progress_apply(calculate_all)
data_filter.head()

  0%|          | 0/2445 [00:00<?, ?it/s]

,ro5_rule,pfizer_rule,gsk_rule,goldentriangle_rule,qed,qed_excellent,sascore,sascore_excellent,fsp3,fsp3_excellent,...,pains_matched_atoms,alarmnmr_accepted,alarmnmr_matched_names,alarmnmr_matched_atoms,bms_accepted,bms_matched_names,bms_matched_atoms,chelator_accepted,chelator_matched_names,chelator_matched_atoms
0,True,False,True,True,0.501767,False,1.891632,True,0.117647,False,...,[-],True,[-],[-],False,[alpha_dicarbonyl],"[((1, 0, 3, 4),)]",True,[-],[-]
1,True,True,False,False,0.388315,False,4.020579,True,0.375000,False,...,[-],False,"[[SH], SCS, c1sccc1]","[((17, 18), (19, 20)), ((16, 17, 18), (20, 19,...",True,[-],[-],True,[-],[-]
2,False,False,False,False,0.229178,False,2.504230,True,0.160000,False,...,[-],False,[C=NN],"[((16, 17, 18),)]",False,[alpha_dicarbonyl],"[((6, 7, 8, 9), (19, 20, 21, 22))]",True,[-],[-]
3,True,False,False,False,0.278585,False,2.224255,True,0.037037,False,...,[-],False,"[c1ccccc1O, [OH]c1ccccc1N]","[((12, 11, 10, 9, 8, 24, 13), (27, 26, 25, 6, ...",True,[-],[-],True,[-],[-]
4,True,False,False,True,0.382177,False,2.345639,True,0.181818,False,...,[-],True,[-],[-],False,[alpha_dicarbonyl],"[((1, 0, 23, 24), (11, 12, 13, 14))]",True,[-],[-]


In [9]:
data = pd.concat([df, data_filter], axis = 1)
data.head()

,ID,Canomical_smiles,Activity,Canomicalsmiles,ro5_rule,pfizer_rule,gsk_rule,goldentriangle_rule,qed,qed_excellent,...,pains_matched_atoms,alarmnmr_accepted,alarmnmr_matched_names,alarmnmr_matched_atoms,bms_accepted,bms_matched_names,bms_matched_atoms,chelator_accepted,chelator_matched_names,chelator_matched_atoms
0,CHEMBL19464,O=C(O)C(=O)CC(=O)c1cccc(Cc2ccccc2)c1,8.00,O=C(O)C(=O)CC(=O)c1cccc(Cc2ccccc2)c1,True,False,True,True,0.501767,False,...,[-],True,[-],[-],False,[alpha_dicarbonyl],"[((1, 0, 3, 4),)]",True,[-],[-]
1,CHEMBL193783,CCc1cc(/C=N/CC/N=C/c2cc(CC)sc2S)c(S)s1,5.22,CCc1cc(/C=N/CC/N=C/c2cc(CC)sc2S)c(S)s1,True,True,False,False,0.388315,False,...,[-],False,"[[SH], SCS, c1sccc1]","[((17, 18), (19, 20)), ((16, 17, 18), (20, 19,...",True,[-],[-],True,[-],[-]
2,CHEMBL191854,Cc1ccc(NC(=O)C(=O)NCc2ccc(/C=N\NC(=O)C(=O)Nc3c...,4.16,Cc1ccc(NC(=O)C(=O)NCc2ccc(/C=N\NC(=O)C(=O)Nc3c...,False,False,False,False,0.229178,False,...,[-],False,[C=NN],"[((16, 17, 18),)]",False,[alpha_dicarbonyl],"[((6, 7, 8, 9), (19, 20, 21, 22))]",True,[-],[-]
3,CHEMBL195104,O=C(Nc1ccc(Cc2ccc(O)c(NC(=O)c3ccccc3F)c2)cc1O)...,4.48,O=C(Nc1ccc(Cc2ccc(O)c(NC(=O)c3ccccc3F)c2)cc1O)...,True,False,False,False,0.278585,False,...,[-],False,"[c1ccccc1O, [OH]c1ccccc1N]","[((12, 11, 10, 9, 8, 24, 13), (27, 26, 25, 6, ...",True,[-],[-],True,[-],[-]
4,CHEMBL195166,O=C(NCc1cccc(CNC(=O)C(=O)NCc2ccco2)c1)C(=O)NCc...,4.40,O=C(NCc1cccc(CNC(=O)C(=O)NCc2ccco2)c1)C(=O)NCc...,True,False,False,True,0.382177,False,...,[-],True,[-],[-],False,[alpha_dicarbonyl],"[((1, 0, 23, 24), (11, 12, 13, 14))]",True,[-],[-]


# 3. Featurizer

In [4]:
from Featurizer import Featurize

reading NP model ...
model in


In [5]:
import warnings
warnings.filterwarnings(action='ignore')
data_name = ['HIV_integrase']


df = pd.read_csv('HIV_integrase.csv')
    #display(df.head(2))
folder = str('./HIV_integrase')
isExist = os.path.exists(folder)
if not isExist:
    os.makedirs(folder)
save_dir = folder+'/'
fp = Featurize(data=df, smile_col='Canomical_smiles', activity_col='Activity', 
            ID='ID', save_dir=save_dir, m2v_path = '../model_300dim.pkl', standardize = True)
fp.Apply()

STANDARDIZING MOLECULES...


  0%|          | 0/2445 [00:00<?, ?it/s]

  0%|          | 0/2377 [00:00<?, ?it/s]

CALCULATING RDK5 FINGERPRINTS...


  0%|          | 0/2377 [00:00<?, ?it/s]

...

CALCULATING Mordred descriptors...


100%|███████████████████████████████████████| 2377/2377 [01:48<00:00, 21.97it/s]


CALCULATING Mol2vec...


  0%|          | 0/2377 [00:00<?, ?it/s]

FINISH CALCULATING!


In [5]:
%pwd

'/home/labhhc/Documents/Project/CADD_package/MolecularClustering/ultility/Test'